In [9]:
import numpy as np
import pandas as pd
import akshare as ak
import dolphindb as ddb
import tushare as ts
import time 
import requests
import datetime
conn = ddb.session()
conn.connect("localhost", 8848, "admin", "123456") # 如果是在 dolphindb-1 上运行, 则需要将 localhost 改为 dolphindb-1
conn.run("""
db=database("dfs://stock");
market_index = loadTable('dfs://stock','market_index');
stock_ma = loadTable('dfs://stock', 'stock_ma');
stock_ind = loadTable('dfs://stock', 'stock_ind');
concept_ind = loadTable('dfs://stock', 'concept_ind');
daily_adj = loadTable('dfs://stock', 'daily_adj')
""")
conn.runFile("update_ddb_tables.dos")

In [5]:
token = "f1ce53736e3b6777425d3df97c05e7460a55534db8ece60114c9e2a3"
ts.set_token(token)
pro = ts.pro_api()

In [6]:
def update_daily_adj(date):
    # 先下载 daily 数据
    # 判断是否已经有date的数据
    conn.run("daily_adj = loadTable('dfs://stock', `daily_adj)")
    if conn.run(f"select 1 from daily_adj where data_dt={date} limit 1").shape[0] == 0:
        print(f"已经存在{date}的daily_adj数据")
    else:
        # 下载数据
        df = pro.daily(start_date=date.replace(".",""), end_date=date.replace(".",""), adj='hfq')
        if df.shape[0] == 0:
            print(f"{date}没有数据")
            return
        df = df.query("not ts_code.str.contains('.BJ')", engine='python')
        # 把chg>20的刷成21, 把chg<-20的刷成-21
        df.pct_chg[df.pct_chg > 21] = 21
        df.pct_chg[df.pct_chg < -21] = -21      
        # to daily_adj
        new_order = ['trade_date', 'ts_code', 'pct_chg', 'open','high', 'low', 'close', 'amount']
        df = df.reindex(columns=new_order)
        df.rename(columns={'trade_date':'data_dt', 'amount':'amt', "pct_chg":"chg"}, inplace=True)

        # 日期从string转成datetime,然后上传.
        df['data_dt'] = pd.to_datetime(df['data_dt'], format='%Y%m%d').values.astype('datetime64[D]')
        df['ts_code'] = df['ts_code'].astype("object")
        conn.upload({'daily_adj_tmp':df})
        conn.run("tableInsert(daily_adj,(select * from daily_adj_tmp))")
        
        print(f"更新完成{date}的daily_adj数据")

update_daily_adj("2023.02.28")

更新完成2023.02.28的daily_adj数据


/var/folders/n1/p3lkv10n0ds8849ms4mn11cw0000gq/T/ipykernel_78003/1098383343.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.pct_chg[df.pct_chg > 21] = 21
/Users/cf/anaconda3/lib/python3.9/site-packages/pandas/core/generic.py:8870: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/var/folders/n1/p3lkv10n0ds8849ms4mn11cw0000gq/T/ipykernel_78003/1098383343.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

In [10]:

# 批量更新daily_adj
def batch_update_daily_adj():
    conn.run("daily_adj = loadTable('dfs://stock', `daily_adj)")
    start_date = np.datetime_as_string(conn.run("select max(data_dt) from daily_adj").values[0][0], unit='D').replace("-",".")
    today = datetime.datetime.today().strftime("%Y.%m.%d")
    end_date = conn.run(f"transFreq({today}, 'SSE')").astype("str").replace("-",".") 
    print(start_date,"---", end_date)
    download_dates = conn.run(f"getMarketCalendar('SSE', {start_date}, {end_date})").tolist()
    
    for date in download_dates:
        date = date.strftime("%Y.%m.%d")
        print(date)
        while True:
            try:
                update_daily_adj(date)
                break
            except requests.exceptions.ConnectTimeout:
                print("出错了, 重试")
                time.sleep(5)
                continue
    print("更新完成批量daily_adj")

batch_update_daily_adj()
      

2023.02.28 --- 2023.06.29
2023.02.28


/var/folders/n1/p3lkv10n0ds8849ms4mn11cw0000gq/T/ipykernel_78003/1098383343.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.pct_chg[df.pct_chg > 21] = 21
/Users/cf/anaconda3/lib/python3.9/site-packages/pandas/core/generic.py:8870: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/var/folders/n1/p3lkv10n0ds8849ms4mn11cw0000gq/T/ipykernel_78003/1098383343.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

更新完成2023.02.28的daily_adj数据
2023.03.01
更新完成2023.03.01的daily_adj数据
2023.03.02
更新完成2023.03.02的daily_adj数据
2023.03.03
出错了, 重试
更新完成2023.03.03的daily_adj数据
2023.03.06
更新完成2023.03.06的daily_adj数据
2023.03.07
更新完成2023.03.07的daily_adj数据
2023.03.08
更新完成2023.03.08的daily_adj数据
2023.03.09
出错了, 重试
更新完成2023.03.09的daily_adj数据
2023.03.10
更新完成2023.03.10的daily_adj数据
2023.03.13
更新完成2023.03.13的daily_adj数据
2023.03.14
更新完成2023.03.14的daily_adj数据
2023.03.15
更新完成2023.03.15的daily_adj数据
2023.03.16
更新完成2023.03.16的daily_adj数据
2023.03.17
更新完成2023.03.17的daily_adj数据
2023.03.20
更新完成2023.03.20的daily_adj数据
2023.03.21
更新完成2023.03.21的daily_adj数据
2023.03.22
更新完成2023.03.22的daily_adj数据
2023.03.23
更新完成2023.03.23的daily_adj数据
2023.03.24
出错了, 重试
更新完成2023.03.24的daily_adj数据
2023.03.27
更新完成2023.03.27的daily_adj数据
2023.03.28
更新完成2023.03.28的daily_adj数据
2023.03.29
更新完成2023.03.29的daily_adj数据
2023.03.30
更新完成2023.03.30的daily_adj数据
2023.03.31
更新完成2023.03.31的daily_adj数据
2023.04.03
更新完成2023.04.03的daily_adj数据
2023.04.04
出错了, 重试
更新完成2023.04.04的dai

In [14]:
conn.run("""
    //update_market_index()
    //update_stock_ma()
    //update_stock_ind()
    update_concept_ind()
""")

In [18]:
conn.run("""
    start_date = 2022.01.01
    end_date = 2022.12.31
    //init_market_index(start_date, end_date)
    init_stock_ma(start_date, end_date)
    init_stock_ind(start_date, end_date)
    init_concept_ind(start_date, end_date)  // concept_ind 需要单独启动一遍, 因为换concept_cons了
    """)

    

In [17]:
# # 热更新
conn.run("""
//update_market_index()
update_stock_ma()
update_stock_ind()
update_concept_ind()
""")

In [12]:
conn.run("basic = loadTable('dfs://stock', 'basic')")
conn.run("select * from basic")

,ts_code,symbol,name,area,industry,market,list_date
0,000001.SZ,000001,平安银行,深圳,银行,主板,19910403
1,000002.SZ,000002,万科A,深圳,全国地产,主板,19910129
2,000004.SZ,000004,ST国华,深圳,软件服务,主板,19910114
3,000005.SZ,000005,ST星源,深圳,环境保护,主板,19901210
4,000006.SZ,000006,深振业A,深圳,区域地产,主板,19920427
...,...,...,...,...,...,...,...
5016,688799.SH,688799,华纳药厂,湖南,化学制药,科创板,20210713
5017,688800.SH,688800,瑞可达,江苏,元器件,科创板,20210722
5018,688819.SH,688819,天能股份,浙江,电气设备,科创板,20210118
5019,688981.SH,688981,中芯国际,上海,半导体,科创板,20200716


In [30]:

import requests
import time


def update_concept_cons(concept):
    conn.run("db = database('dfs://stock');")
    conn.run("basic = loadTable('dfs://stock', 'basic');")
    if  conn.run(f"existsTable('dfs://stock', 'concept_cons')"):
        conn.run("concept_cons = loadTable('dfs://stock', 'concept_cons');")
        if conn.run(f"select * from  concept_cons where 概念名称 = '{concept}'").shape[0] > 0:
            print("已经存在", concept)
            return
    while True:
        try:
            print("开始下载", concept)
            df = ak.stock_board_concept_cons_ths(symbol=concept)
            df['概念名称'] = concept
            df = df[['代码', '名称', '概念名称']]
            conn.upload({"concept_cons_df": df})
            if not conn.run(f"existsTable('dfs://stock', 'concept_cons')"):
                conn.run("""
                t2 = select b.ts_code, c.* from concept_cons_df c
                    left join basic b  on b.symbol = c.代码 
                concept_cons = db.createTable(t2, 'concept_cons');
                concept_cons.append!(select * from t2);
                """)
            else:
                conn.run("concept_cons = loadTable('dfs://stock', 'concept_cons');")
                conn.run(f"""
                delete from concept_cons where 概念名称 = '{concept}';
                t2 = select b.ts_code, c.* from concept_cons_df c
                    left join basic b  on b.symbol = c.代码 
                concept_cons.append!(select * from t2);
                """)
            print("下载完成", concept)
            break
        except (requests.exceptions.ConnectTimeout,ValueError):
            print("出错了, 重试")
            time.sleep(5)
            continue
        

def batch_update_concept_cons():
    
   
    conn.run("concept_list_new = loadTable('dfs://stock', 'concept_list_new');")
    concept_list = conn.run("""select 概念名称 as concept from concept_list_new
    """)['concept'].values
        
    for concept in concept_list:
        update_concept_cons(concept)
    print("更新完成")


batch_update_concept_cons()


已经存在 算力租赁
已经存在 空间计算
已经存在 英伟达概念
已经存在 脑机接口
已经存在 MR（混合现实）
已经存在 同花顺中特估100
已经存在 液冷服务器
已经存在 太赫兹
已经存在 猴痘概念
已经存在 粮食概念
已经存在 存储芯片
已经存在 数据要素
已经存在 烟草
已经存在 首发新股
已经存在 一季报预增
已经存在 MLOps概念
已经存在 电力物联网
已经存在 ERP概念
已经存在 DRG/DIP
已经存在 超导概念
已经存在 6G概念
已经存在 时空大数据
已经存在 毫米波雷达
已经存在 数字水印
已经存在 共封装光学（CPO）
已经存在 固态电池
已经存在 ChatGPT概念
已经存在 成飞概念
已经存在 国产航母
已经存在 蒙脱石散
已经存在 血氧仪
已经存在 新冠特效药
已经存在 POE胶膜
已经存在 熊去氧胆酸
已经存在 抗病毒面料
已经存在 数据确权
已经存在 抗原检测
已经存在 证金持股
已经存在 国企改革
已经存在 深圳国企改革
已经存在 上海国企改革
已经存在 央企国企改革
已经存在 PET铜箔
已经存在 AIGC概念
已经存在 高压氧舱
已经存在 Web3.0
已经存在 创新药
已经存在 信创
已经存在 有机硅概念
已经存在 空气能热泵
已经存在 先进封装（Chiplet）
已经存在 减速器
已经存在 TOPCON电池
已经存在 钙钛矿电池
已经存在 生物质能发电
已经存在 一体化压铸
已经存在 F5G概念
已经存在 比亚迪概念
已经存在 超超临界发电
已经存在 方舱医院
已经存在 噪声防治
已经存在 露营经济
已经存在 肝炎概念
已经存在 统一大市场
已经存在 MicroLED概念
已经存在 毛发医疗
已经存在 恒大概念
已经存在 疫情监测
已经存在 华为欧拉
已经存在 家庭医生
已经存在 华为鲲鹏
已经存在 低辐射玻璃（Low-E）
已经存在 国资云
已经存在 电子身份证
已经存在 数字经济
已经存在 金属铅
已经存在 托育服务
已经存在 跨境支付（CIPS）
已经存在 中俄贸易概念
已经存在 俄乌冲突概念
已经存在 东数西算（算力）
已经存在 智慧灯杆
已经存在 土壤修复
已经存在 民爆概念
已经存在 股权转让
已经存在 PCB概念
已经存在 抖音概念
已经存在 硅能源
已经存在 重组蛋白
已经存在 新冠治疗
已经存在 电子纸


In [26]:
conn.run("db = database('dfs://stock'); dropTable(db, 'concept_cons')")

In [3]:
import tushare as ts
token = "f1ce53736e3b6777425d3df97c05e7460a55534db8ece60114c9e2a3"
ts.set_token(token)
pro = ts.pro_api()